This is the notebook related to our feature extraction phase. At the end, we obtain two different new .tsv files, one of which will be used in the following steps (*Pre-Processing*).

# Feature Extraction

We expand the feature selection phase by extracting additional features from the data to complement those already present in the *calc_features.py* and *calc_3di.py* scripts. This process aims to obtain features that are both meaningful and informative, while also carefully balancing computational effort and memory usage.

We do not directly change the content of the two scripts. Rather, we extract new information from them through this notebook.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install biopython

In [ ]:
from Bio.PDB import PDBList
from Bio.PDB.PDBParser import PDBParser
import zipfile
import pandas as pd
import os
import numpy as np

In [ ]:
# Set Pandas to display all columns
pd.set_option('display.max_columns', None)

## Data Loading

We load the dataset from the *features_ring.zip* archive, which contains multiple .tsv files with tab-separated values.

We use a generator expression to read and concatenate these files on-the-fly, which avoids building large intermediate lists in memory and is more efficient.
The result is a single combined DataFrame containing all the data from the .tsv files.

In [ ]:
# Specify your folder path
folder_path = '/content/drive/MyDrive/Corsi del Semestre/STRUCTURAL BIOINFORMATICS/Structural Bioinfo PROJECT'
# Path to the ZIP file
zip_path = f'{folder_path}/features_ring.zip'

# Read ZIP and use generator expression -> It generates elements one at a time on demand (lazy evaluation), instead of building a complete list in memory right away.
with zipfile.ZipFile(zip_path) as zip_ref: # filter out the tsv.file
    tsv_files = [f for f in zip_ref.namelist() if f.endswith('.tsv')]

    combined_df = pd.concat( # generator is used to read the file in time
        (pd.read_csv(zip_ref.open(f), sep='\t') for f in tsv_files),
        ignore_index=True
    )

In [ ]:
combined_df.shape

(2968986, 32)

In [ ]:
combined_df.columns

Index(['pdb_id', 's_ch', 's_resi', 's_ins', 's_resn', 's_ss8', 's_rsa',
       's_phi', 's_psi', 's_a1', 's_a2', 's_a3', 's_a4', 's_a5', 's_3di_state',
       's_3di_letter', 't_ch', 't_resi', 't_ins', 't_resn', 't_ss8', 't_rsa',
       't_phi', 't_psi', 't_a1', 't_a2', 't_a3', 't_a4', 't_a5', 't_3di_state',
       't_3di_letter', 'Interaction'],
      dtype='object')

In [ ]:
combined_df.head()

,pdb_id,s_ch,s_resi,s_ins,s_resn,s_ss8,s_rsa,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,s_3di_state,s_3di_letter,t_ch,t_resi,t_ins,t_resn,t_ss8,t_rsa,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5,t_3di_state,t_3di_letter,Interaction
0,1b0y,A,28,,R,H,0.056,-1.127,-0.711,1.538,-0.055,1.502,0.440,2.897,13.0,N,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,VDW
1,1b0y,A,27,,E,S,0.531,-1.622,0.483,1.357,-1.453,1.477,0.113,-0.837,1.0,B,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,HBOND
2,1b0y,A,47,,Q,T,0.460,-0.986,-0.566,0.931,-0.179,-3.005,-0.503,-1.853,13.0,N,A,84,,A,P,0.472,-1.557,2.544,-0.591,-1.302,-0.733,1.570,-0.146,2.0,C,HBOND
3,1b0y,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,A,45,,N,G,0.274,-1.941,0.412,0.945,0.828,1.299,-0.169,0.933,13.0,N,HBOND
4,1b0y,A,37,,P,-,0.551,-0.880,2.320,0.189,2.081,-1.628,0.421,-1.392,14.0,O,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,HBOND


## New features
We create a copy of the original data where we apply the modifications.


In [ ]:
combined_new_df = combined_df.copy()

### Same chain

With the following lines we explore for the chain ID features, the different discrete values they can assume.

In [ ]:
chain_cols = ['s_ch', 't_ch']

# Convert specified columns to 'category' dtype
combined_df[chain_cols] = combined_df[chain_cols].astype('category')

In [ ]:
for col in combined_df.select_dtypes(include='category').columns:
    print(f"{col}: {combined_df[col].cat.categories.tolist()}")

s_ch: ['0', '1', '2', '3', '4', '5', '6', '7', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
t_ch: ['0', '1', '2', '3', '4', '5', '6', '7', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


We notice that these two features (one for each residue in the pair) have 60 different values, which might not carry much meaningful information.

In [ ]:
combined_df[['s_ch', 't_ch']].describe()

,s_ch,t_ch
count,2968986,2968986
unique,60,60
top,A,A
freq,1766351,1689125


 Thus, we explore some alternative ways to represent these features and make use of the information relative to the chain.

First, we explore how many interactions occur between residues in the same chain (i.e., *intra-chain*) or in different chains (i.e., *inter-chain*).

In [ ]:
num_diff = (combined_df['s_ch'] != combined_df['t_ch']).sum()
print(f"Number of rows where s_ch != t_ch: {num_diff}")

Number of rows where s_ch != t_ch: 125524


As we can see, more than 100K interactions are inter-chain, over a total of almost 3M interactions, meaning that most of them are intra-chain.

In [ ]:
perc_diff = (num_diff / combined_df.shape[0]) * 100
print(f"Percentage of rows where s_ch != t_ch: {perc_diff:.2f}%")

Percentage of rows where s_ch != t_ch: 4.23%


To further investigate these variables, we look into their distribution across the various interaction types.

In [ ]:
# Total number of rows per Interaction type
total_counts = combined_df.groupby('Interaction').size()

# Number of rows where s_ch ≠ t_ch per Interaction type
diff_counts = combined_df[combined_df['s_ch'] != combined_df['t_ch']].groupby('Interaction').size()

# Compute percentage
diff_percent = (diff_counts / total_counts * 100).round(2)

# Combine into a summary DataFrame
summary = pd.DataFrame({
    'Total': total_counts,
    's_ch ≠ t_ch': diff_counts,
    '% Different': diff_percent
}).fillna(0).astype({'Total': 'int', 's_ch ≠ t_ch': 'int', '% Different': 'float'})

print(summary)

               Total  s_ch ≠ t_ch  % Different
Interaction                                   
HBOND        1055929        31047         2.94
IONIC          35391         3576        10.10
PICATION        8885          662         7.45
PIHBOND         1790          128         7.15
PIPISTACK      38283         2225         5.81
SSBOND          2100           58         2.76
VDW           737061        29740         4.03


As shown by the data, no specific interaction type appears to be significantly more frequent in inter-chain contacts compared to intra-chain ones.

Given the relatively low number of inter-chain interactions and the fact that chain ID values carry no inherent structural or physical meaning - being arbitrarily assigned across different structures - we introduce a new variable: *same_chain*.

This boolean feature simply indicates whether the interaction occurs within the same chain (TRUE) or between different chains (FALSE), and it entirely replaces the original chain ID variables (*s_ch* and *t_ch*).

While this choice results in the loss of information about which specific chains are interacting, this detail is considered negligible due to the rarity of inter-chain interactions. Moreover, modeling individual chain identifiers would provide little value, as they are not consistent or interpretable across structures.

In [ ]:
# Create same_chain feature: 1 if s_ch == t_ch, else 0
combined_new_df['same_chain'] = (combined_new_df['s_ch'] == combined_new_df['t_ch']).astype(int)

combined_df['same_chain'] = (combined_df['s_ch'] == combined_df['t_ch']).astype(int)

Therefore, the **same_chain** feature allows us to distinguish between intra-chain and inter-chain interactions.

This is also useful from a biological point of view:

- same_chain == 1 → the interaction is more likely to be an HBOND or VDW interaction;

- same_chain == 0 → the interaction is more likely to be an IONIC interaction, a disulfide bridge (SSBOND), or a π-π stacking interaction (PIPISTACK).

In [ ]:
combined_new_df.shape

(2968986, 33)

In [ ]:
combined_df.shape

(2968986, 33)

In [ ]:
combined_new_df.columns

Index(['pdb_id', 's_ch', 's_resi', 's_ins', 's_resn', 's_ss8', 's_rsa',
       's_phi', 's_psi', 's_a1', 's_a2', 's_a3', 's_a4', 's_a5', 's_3di_state',
       's_3di_letter', 't_ch', 't_resi', 't_ins', 't_resn', 't_ss8', 't_rsa',
       't_phi', 't_psi', 't_a1', 't_a2', 't_a3', 't_a4', 't_a5', 't_3di_state',
       't_3di_letter', 'Interaction', 'same_chain'],
      dtype='object')

In [ ]:
combined_new_df.columns

Index(['pdb_id', 's_ch', 's_resi', 's_ins', 's_resn', 's_ss8', 's_rsa',
       's_phi', 's_psi', 's_a1', 's_a2', 's_a3', 's_a4', 's_a5', 's_3di_state',
       's_3di_letter', 't_ch', 't_resi', 't_ins', 't_resn', 't_ss8', 't_rsa',
       't_phi', 't_psi', 't_a1', 't_a2', 't_a3', 't_a4', 't_a5', 't_3di_state',
       't_3di_letter', 'Interaction', 'same_chain'],
      dtype='object')

In [ ]:
combined_new_df.head()

,pdb_id,s_ch,s_resi,s_ins,s_resn,s_ss8,s_rsa,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,s_3di_state,s_3di_letter,t_ch,t_resi,t_ins,t_resn,t_ss8,t_rsa,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5,t_3di_state,t_3di_letter,Interaction,same_chain
0,1b0y,A,28,,R,H,0.056,-1.127,-0.711,1.538,-0.055,1.502,0.440,2.897,13.0,N,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,VDW,1
1,1b0y,A,27,,E,S,0.531,-1.622,0.483,1.357,-1.453,1.477,0.113,-0.837,1.0,B,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,HBOND,1
2,1b0y,A,47,,Q,T,0.460,-0.986,-0.566,0.931,-0.179,-3.005,-0.503,-1.853,13.0,N,A,84,,A,P,0.472,-1.557,2.544,-0.591,-1.302,-0.733,1.570,-0.146,2.0,C,HBOND,1
3,1b0y,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,A,45,,N,G,0.274,-1.941,0.412,0.945,0.828,1.299,-0.169,0.933,13.0,N,HBOND,1
4,1b0y,A,37,,P,-,0.551,-0.880,2.320,0.189,2.081,-1.628,0.421,-1.392,14.0,O,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,HBOND,1


In [ ]:
combined_df.head()

,pdb_id,s_ch,s_resi,s_ins,s_resn,s_ss8,s_rsa,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,s_3di_state,s_3di_letter,t_ch,t_resi,t_ins,t_resn,t_ss8,t_rsa,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5,t_3di_state,t_3di_letter,Interaction,same_chain
0,1b0y,A,28,,R,H,0.056,-1.127,-0.711,1.538,-0.055,1.502,0.440,2.897,13.0,N,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,VDW,1
1,1b0y,A,27,,E,S,0.531,-1.622,0.483,1.357,-1.453,1.477,0.113,-0.837,1.0,B,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,HBOND,1
2,1b0y,A,47,,Q,T,0.460,-0.986,-0.566,0.931,-0.179,-3.005,-0.503,-1.853,13.0,N,A,84,,A,P,0.472,-1.557,2.544,-0.591,-1.302,-0.733,1.570,-0.146,2.0,C,HBOND,1
3,1b0y,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,A,45,,N,G,0.274,-1.941,0.412,0.945,0.828,1.299,-0.169,0.933,13.0,N,HBOND,1
4,1b0y,A,37,,P,-,0.551,-0.880,2.320,0.189,2.081,-1.628,0.421,-1.392,14.0,O,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,HBOND,1


### Delta RSA

The **delta_rsa** feature represents the absolute difference in relative solvent accessibility (RSA) between the two contacting residues. This feature can help improve the discrimination between different types of interactions.

In general:

- Small difference in RSA →  may indicate situations where residues that are both buried (low RSA) or both exposed (high RSA);

- Large difference in RSA → may indicate unusual or asymmetric contact patterns.

In [ ]:
def add_delta_rsa(df):
    df['delta_rsa'] = abs(df['s_rsa'] - df['t_rsa'])
    return df

In [ ]:
combined_new_df = add_delta_rsa(combined_new_df)
combined_new_df.shape

(2968986, 34)

In [ ]:
combined_new_df.columns

Index(['pdb_id', 's_ch', 's_resi', 's_ins', 's_resn', 's_ss8', 's_rsa',
       's_phi', 's_psi', 's_a1', 's_a2', 's_a3', 's_a4', 's_a5', 's_3di_state',
       's_3di_letter', 't_ch', 't_resi', 't_ins', 't_resn', 't_ss8', 't_rsa',
       't_phi', 't_psi', 't_a1', 't_a2', 't_a3', 't_a4', 't_a5', 't_3di_state',
       't_3di_letter', 'Interaction', 'same_chain', 'delta_rsa'],
      dtype='object')

In [ ]:
combined_new_df.head()

,pdb_id,s_ch,s_resi,s_ins,s_resn,s_ss8,s_rsa,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,s_3di_state,s_3di_letter,t_ch,t_resi,t_ins,t_resn,t_ss8,t_rsa,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5,t_3di_state,t_3di_letter,Interaction,same_chain,delta_rsa
0,1b0y,A,28,,R,H,0.056,-1.127,-0.711,1.538,-0.055,1.502,0.440,2.897,13.0,N,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,VDW,1,0.378
1,1b0y,A,27,,E,S,0.531,-1.622,0.483,1.357,-1.453,1.477,0.113,-0.837,1.0,B,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,HBOND,1,0.097
2,1b0y,A,47,,Q,T,0.460,-0.986,-0.566,0.931,-0.179,-3.005,-0.503,-1.853,13.0,N,A,84,,A,P,0.472,-1.557,2.544,-0.591,-1.302,-0.733,1.570,-0.146,2.0,C,HBOND,1,0.012
3,1b0y,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,A,45,,N,G,0.274,-1.941,0.412,0.945,0.828,1.299,-0.169,0.933,13.0,N,HBOND,1,0.272
4,1b0y,A,37,,P,-,0.551,-0.880,2.320,0.189,2.081,-1.628,0.421,-1.392,14.0,O,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,HBOND,1,0.005


### Delta Atchley factors
Atchley factors are five numerical values that summarize the physicochemical properties of amino acids (such as hydrophobicity, charge, etc.).

The **delta_atchley** features represent the absolute differences between the Atchley factors of the two contacting residues.

Our intuition is that this type of information allows for better discrimination between different classes of contacts. Indeed, residues with similar Atchley profiles tend to form certain types of interactions, while large differences may indicate other types of interactions.

In [ ]:
def add_delta_atchley(df):
    for i in range(1, 6):
        df[f'delta_atchley_{i}'] = (df[f's_a{i}'] - df[f't_a{i}']).abs()
    return df

In [ ]:
combined_new_df = add_delta_atchley(combined_new_df)
combined_new_df.shape

(2968986, 39)

In [ ]:
combined_new_df.columns

Index(['pdb_id', 's_ch', 's_resi', 's_ins', 's_resn', 's_ss8', 's_rsa',
       's_phi', 's_psi', 's_a1', 's_a2', 's_a3', 's_a4', 's_a5', 's_3di_state',
       's_3di_letter', 't_ch', 't_resi', 't_ins', 't_resn', 't_ss8', 't_rsa',
       't_phi', 't_psi', 't_a1', 't_a2', 't_a3', 't_a4', 't_a5', 't_3di_state',
       't_3di_letter', 'Interaction', 'same_chain', 'delta_rsa',
       'delta_atchley_1', 'delta_atchley_2', 'delta_atchley_3',
       'delta_atchley_4', 'delta_atchley_5'],
      dtype='object')

In [ ]:
combined_new_df.head()

,pdb_id,s_ch,s_resi,s_ins,s_resn,s_ss8,s_rsa,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,s_3di_state,s_3di_letter,t_ch,t_resi,t_ins,t_resn,t_ss8,t_rsa,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5,t_3di_state,t_3di_letter,Interaction,same_chain,delta_rsa,delta_atchley_1,delta_atchley_2,delta_atchley_3,delta_atchley_4,delta_atchley_5
0,1b0y,A,28,,R,H,0.056,-1.127,-0.711,1.538,-0.055,1.502,0.440,2.897,13.0,N,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,VDW,1,0.378,2.129,1.247,2.235,1.130,3.043
1,1b0y,A,27,,E,S,0.531,-1.622,0.483,1.357,-1.453,1.477,0.113,-0.837,1.0,B,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,HBOND,1,0.097,1.948,0.151,2.210,1.457,0.691
2,1b0y,A,47,,Q,T,0.460,-0.986,-0.566,0.931,-0.179,-3.005,-0.503,-1.853,13.0,N,A,84,,A,P,0.472,-1.557,2.544,-0.591,-1.302,-0.733,1.570,-0.146,2.0,C,HBOND,1,0.012,1.522,1.123,2.272,2.073,1.707
3,1b0y,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,A,45,,N,G,0.274,-1.941,0.412,0.945,0.828,1.299,-0.169,0.933,13.0,N,HBOND,1,0.272,0.412,2.281,0.178,0.282,1.770
4,1b0y,A,37,,P,-,0.551,-0.880,2.320,0.189,2.081,-1.628,0.421,-1.392,14.0,O,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,HBOND,1,0.005,1.168,3.534,3.105,0.308,0.555


### CA distance
This is a fundamental geometric feature. Indeed, the **CA distance** provides a good approximation of the physical distance between two residues.

In particular:
- Residues that are far apart cannot form weak interactions such as VDW or HBOND;

- Certain interactions, like stacking (PIPISTACK) or disulfide bridges (SSBOND), require the residues to be within an optimal distance range to form stable contacts.

In [ ]:
pdb_ids = combined_new_df['pdb_id'].unique()
print(pdb_ids)

['1b0y' '1bkr' '1bs9' ... '9h3e' '9h48' '9in7']


In [ ]:
pdb_ids_raw = combined_new_df['pdb_id'].dropna().astype(str) # elimination of NaN values and conversion in string type
total_ids_before = pdb_ids_raw.str.strip().str.lower().nunique()

# Fiter out not valid values
pdb_ids = pdb_ids_raw.str.strip()              # covertion of the string and elimination of the blank spaces
pdb_ids = pdb_ids[pdb_ids.str.len() == 4]      # only lenght of 4
pdb_ids = pdb_ids[pdb_ids.str.isalnum()]       # only alphanumeric
pdb_ids = pdb_ids.str.lower().unique()         # unique and lowercase

print(f"Found {len(pdb_ids)} valid pdb ids.")
print(f"Found {total_ids_before - len(pdb_ids)} invalid pdb ids.")

pdbl = PDBList()

for pdb_id in pdb_ids:
    #print(f"Downloading {pdb_id}...")
    pdbl.retrieve_pdb_file(pdb_id, pdir='pdb_files', file_format='pdb')

# 12 not valid id
# 3902  valid pdb id  ->  not alble to load 19 structures

Found 3902 valid pdb ids.
Found 12 invalid pdb ids.
Structure exists: 'pdb_files/pdb1b0y.ent' 
Structure exists: 'pdb_files/pdb1bkr.ent' 
Structure exists: 'pdb_files/pdb1bs9.ent' 
Structure exists: 'pdb_files/pdb1byi.ent' 
Structure exists: 'pdb_files/pdb1c0p.ent' 
Structure exists: 'pdb_files/pdb1dj0.ent' 
Structure exists: 'pdb_files/pdb1ekq.ent' 
Structure exists: 'pdb_files/pdb1es5.ent' 
Structure exists: 'pdb_files/pdb1es9.ent' 
Structure exists: 'pdb_files/pdb1f7l.ent' 
Structure exists: 'pdb_files/pdb1fg7.ent' 
Structure exists: 'pdb_files/pdb1fm0.ent' 
Structure exists: 'pdb_files/pdb1fmk.ent' 
Structure exists: 'pdb_files/pdb1fo9.ent' 
Structure exists: 'pdb_files/pdb1g5a.ent' 
Structure exists: 'pdb_files/pdb1g69.ent' 
Structure exists: 'pdb_files/pdb1ga6.ent' 
Structure exists: 'pdb_files/pdb1gcu.ent' 
Structure exists: 'pdb_files/pdb1gk9.ent' 
Structure exists: 'pdb_files/pdb1gmw.ent' 
Structure exists: 'pdb_files/pdb1gqi.ent' 
Structure exists: 'pdb_files/pdb1gu2.ent' 
St

In [ ]:
def get_ca_coords(pdb_file):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)

    ca_coords = {}

    for model in structure:
        for chain in model:
            for residue in chain:
                if 'CA' in residue:
                    ca = residue['CA'].get_coord()
                    res_id = residue.get_id()
                    key = (chain.id, res_id[1], res_id[2].strip())  # .strip(): remove the eventually blanck space at the end of the string
                    ca_coords[key] = ca
    return ca_coords

In [ ]:
def calculate_ca_distance(row, ca_coords_dict):
    s_key = (row['s_ch'], row['s_resi'], row['s_ins'].strip())
    t_key = (row['t_ch'], row['t_resi'], row['t_ins'].strip())

    if s_key in ca_coords_dict and t_key in ca_coords_dict:
        dist = np.linalg.norm(ca_coords_dict[s_key] - ca_coords_dict[t_key])  # euclidean distance
        return dist
    else:
        return np.nan

After the two cells above, it is possible to observe that, among the total of 3,914 PDB entries:

- 12 have semantic irregularities in their PDB IDs and are therefore discarded;

- Among the valid ones (3,902), 19 do not have downloadable structures via Biopython and therefore cannot be further processed.

In [ ]:
combined_new_df['ca_distance'] = None # new features

for pdb_id in pdb_ids:
    pdb_file = os.path.join('pdb_files', f'pdb{pdb_id}.ent')

    if not os.path.exists(pdb_file):
        print(f"File not found for PDB ID: {pdb_id}, skipping...") # i.e. the not existing structures -> 19
        continue

    ca_coords = get_ca_coords(pdb_file)

    mask = combined_new_df['pdb_id'] == pdb_id # Select all the rows with the current pdb (boolean value)

    # Calculation of the distance for every selected row based on the AC coordinates
    combined_new_df.loc[mask, 'ca_distance'] = combined_new_df.loc[mask].apply(
        lambda row: calculate_ca_distance(row, ca_coords), axis=1
    )

File not found for PDB ID: 4v4m, skipping...
File not found for PDB ID: 7uc3, skipping...
File not found for PDB ID: 7w1c, skipping...
File not found for PDB ID: 8rg1, skipping...
File not found for PDB ID: 9ftc, skipping...
File not found for PDB ID: 8rqp, skipping...
File not found for PDB ID: 8v9b, skipping...
File not found for PDB ID: 8yhf, skipping...
File not found for PDB ID: 8ys9, skipping...
File not found for PDB ID: 9c0y, skipping...
File not found for PDB ID: 9gk6, skipping...
File not found for PDB ID: 8rkr, skipping...
File not found for PDB ID: 8rvf, skipping...
File not found for PDB ID: 8vqx, skipping...
File not found for PDB ID: 9doj, skipping...
File not found for PDB ID: 9gio, skipping...
File not found for PDB ID: 9o8y, skipping...
File not found for PDB ID: 7vub, skipping...
File not found for PDB ID: 8s77, skipping...


In [ ]:
combined_new_df.shape

(2968986, 40)

In [ ]:
combined_new_df.columns

Index(['pdb_id', 's_ch', 's_resi', 's_ins', 's_resn', 's_ss8', 's_rsa',
       's_phi', 's_psi', 's_a1', 's_a2', 's_a3', 's_a4', 's_a5', 's_3di_state',
       's_3di_letter', 't_ch', 't_resi', 't_ins', 't_resn', 't_ss8', 't_rsa',
       't_phi', 't_psi', 't_a1', 't_a2', 't_a3', 't_a4', 't_a5', 't_3di_state',
       't_3di_letter', 'Interaction', 'same_chain', 'delta_rsa',
       'delta_atchley_1', 'delta_atchley_2', 'delta_atchley_3',
       'delta_atchley_4', 'delta_atchley_5', 'ca_distance'],
      dtype='object')

In [ ]:
combined_new_df.head()

,pdb_id,s_ch,s_resi,s_ins,s_resn,s_ss8,s_rsa,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,s_3di_state,s_3di_letter,t_ch,t_resi,t_ins,t_resn,t_ss8,t_rsa,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5,t_3di_state,t_3di_letter,Interaction,same_chain,delta_rsa,delta_atchley_1,delta_atchley_2,delta_atchley_3,delta_atchley_4,delta_atchley_5,ca_distance
0,1b0y,A,28,,R,H,0.056,-1.127,-0.711,1.538,-0.055,1.502,0.440,2.897,13.0,N,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,VDW,1,0.378,2.129,1.247,2.235,1.130,3.043,4.967376
1,1b0y,A,27,,E,S,0.531,-1.622,0.483,1.357,-1.453,1.477,0.113,-0.837,1.0,B,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,HBOND,1,0.097,1.948,0.151,2.210,1.457,0.691,6.062192
2,1b0y,A,47,,Q,T,0.460,-0.986,-0.566,0.931,-0.179,-3.005,-0.503,-1.853,13.0,N,A,84,,A,P,0.472,-1.557,2.544,-0.591,-1.302,-0.733,1.570,-0.146,2.0,C,HBOND,1,0.012,1.522,1.123,2.272,2.073,1.707,6.714707
3,1b0y,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,A,45,,N,G,0.274,-1.941,0.412,0.945,0.828,1.299,-0.169,0.933,13.0,N,HBOND,1,0.272,0.412,2.281,0.178,0.282,1.770,8.626575
4,1b0y,A,37,,P,-,0.551,-0.880,2.320,0.189,2.081,-1.628,0.421,-1.392,14.0,O,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,HBOND,1,0.005,1.168,3.534,3.105,0.308,0.555,5.669582


### 3Di centroid coordinates
We implemented four additional features (two for each residue in the pair) containing the cartesian coordinates (x, y) of the centroids provided in the *states.txt* file.

Since the 3di_state variable represents the index (ranging from 0 to 19) of a 3D structural cluster, we can map each index to its corresponding **centroid coordinates**. This allows us to replace the discrete categorical variable with continuous numerical features that more accurately capture spatial relationships.

In our opinion, this approach brings several advantages:
- Preserves spatial information, since the coordinates reflect the real position in the 3D clustering space;

- Reduces dataset dimensionality, instead of applying OHE to 20 possible states (which would result in 20 sparse features), we use only 2 dense numerical features (for which OHE is not necessary), saving memory and improving computational efficiency.

In [ ]:
# Path to the ZIP file
folder = folder_path + '/classification_ring/3di_model'
centroids = np.loadtxt(f"{folder}/states.txt")

print(centroids.shape)
centroids

(20, 2)


array([[-1.07291007, -0.35998434],
       [-0.13563171, -1.89137328],
       [ 0.49482635, -0.42048582],
       [-0.98744941,  0.81276411],
       [-1.66214275, -0.42586824],
       [ 2.13942504,  0.04861213],
       [ 1.55578887, -0.15026288],
       [ 2.91793561,  1.14372849],
       [-2.88137507,  0.99563217],
       [-1.14000058, -2.00682235],
       [ 3.20252109,  1.73564065],
       [ 1.77688396, -1.30372393],
       [ 0.69011408, -1.25542247],
       [-1.10611844, -1.33966064],
       [ 2.1495142 , -0.80299157],
       [ 2.30597854, -1.49881566],
       [ 2.55217505,  0.60462159],
       [ 0.77863103, -2.16599894],
       [-2.3030262 ,  0.38133013],
       [ 1.02900362,  0.87718374]])

In [ ]:
# To manage possible NAN values
def map_centroid(coord_index, axis):
    try:
        return centroids[int(coord_index), axis]
    except (ValueError, TypeError, IndexError):
        return np.nan

# NEW DATA
combined_new_df['s_centroid_x'] = combined_new_df['s_3di_state'].apply(lambda i: map_centroid(i, 0))
combined_new_df['s_centroid_y'] = combined_new_df['s_3di_state'].apply(lambda i: map_centroid(i, 1))
combined_new_df['t_centroid_x'] = combined_new_df['t_3di_state'].apply(lambda i: map_centroid(i, 0))
combined_new_df['t_centroid_y'] = combined_new_df['t_3di_state'].apply(lambda i: map_centroid(i, 1))


In [ ]:
combined_new_df.shape

(2968986, 44)

In [ ]:
combined_new_df.columns

Index(['pdb_id', 's_ch', 's_resi', 's_ins', 's_resn', 's_ss8', 's_rsa',
       's_phi', 's_psi', 's_a1', 's_a2', 's_a3', 's_a4', 's_a5', 's_3di_state',
       's_3di_letter', 't_ch', 't_resi', 't_ins', 't_resn', 't_ss8', 't_rsa',
       't_phi', 't_psi', 't_a1', 't_a2', 't_a3', 't_a4', 't_a5', 't_3di_state',
       't_3di_letter', 'Interaction', 'same_chain', 'delta_rsa',
       'delta_atchley_1', 'delta_atchley_2', 'delta_atchley_3',
       'delta_atchley_4', 'delta_atchley_5', 'ca_distance', 's_centroid_x',
       's_centroid_y', 't_centroid_x', 't_centroid_y'],
      dtype='object')

In [ ]:
combined_new_df.head()

,pdb_id,s_ch,s_resi,s_ins,s_resn,s_ss8,s_rsa,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,s_3di_state,s_3di_letter,t_ch,t_resi,t_ins,t_resn,t_ss8,t_rsa,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5,t_3di_state,t_3di_letter,Interaction,same_chain,delta_rsa,delta_atchley_1,delta_atchley_2,delta_atchley_3,delta_atchley_4,delta_atchley_5,ca_distance,s_centroid_x,s_centroid_y,t_centroid_x,t_centroid_y
0,1b0y,A,28,,R,H,0.056,-1.127,-0.711,1.538,-0.055,1.502,0.440,2.897,13.0,N,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,VDW,1,0.378,2.129,1.247,2.235,1.130,3.043,4.967376,-1.106118,-1.339661,-1.140001,-2.006822
1,1b0y,A,27,,E,S,0.531,-1.622,0.483,1.357,-1.453,1.477,0.113,-0.837,1.0,B,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,HBOND,1,0.097,1.948,0.151,2.210,1.457,0.691,6.062192,-0.135632,-1.891373,-1.140001,-2.006822
2,1b0y,A,47,,Q,T,0.460,-0.986,-0.566,0.931,-0.179,-3.005,-0.503,-1.853,13.0,N,A,84,,A,P,0.472,-1.557,2.544,-0.591,-1.302,-0.733,1.570,-0.146,2.0,C,HBOND,1,0.012,1.522,1.123,2.272,2.073,1.707,6.714707,-1.106118,-1.339661,0.494826,-0.420486
3,1b0y,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,A,45,,N,G,0.274,-1.941,0.412,0.945,0.828,1.299,-0.169,0.933,13.0,N,HBOND,1,0.272,0.412,2.281,0.178,0.282,1.770,8.626575,0.690114,-1.255422,-1.106118,-1.339661
4,1b0y,A,37,,P,-,0.551,-0.880,2.320,0.189,2.081,-1.628,0.421,-1.392,14.0,O,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,HBOND,1,0.005,1.168,3.534,3.105,0.308,0.555,5.669582,2.149514,-0.802992,0.690114,-1.255422


# Data saving
We save two separate .tsv files to be used in the following pre-processing steps:
- *combinated_df_origin*: contains the features obtained using the original *calc_features.py* and *calc_3di.py* scripts and the *same_chain* feature;

- *combinated_df_new*: contains the original features and all the new features computed above.

This diversification is done in order to observe the differences in terms of models performance based on the feature provided as input.

In [ ]:
def save_df_as_tsv(df, folder, filename):
    os.makedirs(folder, exist_ok=True)
    if not filename.endswith('.tsv'):
        filename += '.tsv'
    path = os.path.join(folder, filename)
    df.to_csv(path, sep='\t', index=False)
    print(f"Saved TSV to: {path}")

In [ ]:
# Set folder and filenames
folder = folder_path + "/datasets"
filename_origin = 'combinated_df_origin'
filename_new = 'combinated_df_new'

# Save
save_df_as_tsv(combined_df, folder, filename_origin)
save_df_as_tsv(combined_new_df, folder, filename_new)

Saved TSV to: /content/drive/MyDrive/Corsi del Semestre/STRUCTURAL BIOINFORMATICS/Structural Bioinfo PROJECT/datasets/combinated_df_origin.tsv
Saved TSV to: /content/drive/MyDrive/Corsi del Semestre/STRUCTURAL BIOINFORMATICS/Structural Bioinfo PROJECT/datasets/combinated_df_new.tsv


All features that, after this phase of feature analysis, are deemed redundant or insignificant are removed during *Pre-Processing*.